# OVA (One versus All Classifier)

### could have used scikit learn OneVsRestClassifier but was getting issues with limited memmory in collab and pc

In [ ]:
!pip install emnist

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras import layers,models
import numpy as np
from emnist import extract_training_samples, extract_test_samples
from sklearn.model_selection import train_test_split

In [ ]:
images, labels = extract_training_samples('bymerge')
testImg,testLabels = extract_test_samples('bymerge')

In [ ]:
print(np.unique(labels))
print(len(labels))
print(images[0].shape)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def create_model():

    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:

alphabets = [chr(i) for i in range(ord('A'),ord('Z')+1)]
alphabets = alphabets+[chr(i) for i in range(ord('a'),ord('z')+1)]
datasetmap = [i for i in range(10)]+alphabets
print(datasetmap)

# Intuition
### there is 62 models here divided into 3 cluster of models
### cluster 1 => handles A-Z
### cluster 2 => handles a-z
### cluster 3 => handles 1-9

### each model predicts if a image belongs of certain class
### example modelA in cluster 1 => predicts if the image is A(1) or not(0)


# change the wanted class and unwanted class for training each model cluster

In [ ]:
indexofallclass = [i for i in range(62)]
wantedclass = [i for i in range(10)]
unwanted = [i for i in range(10,48)]
from keras import backend as K

for i in wantedclass:

  normal_class = i

  print("model",datasetmap[normal_class],"training",normal_class)
  indices_of_normal_class = np.where(labels == normal_class)[0]

  X_desired_class = images[indices_of_normal_class]
  y_desired_class = np.ones(len(indices_of_normal_class))

  indices_of_other_class = np.where((labels != normal_class) & ~np.isin(labels, unwanted))[0]


  X_other_class = images[indices_of_other_class]
  y_other_class = np.zeros(len(indices_of_other_class))

  combined_data = np.concatenate((X_desired_class, X_other_class))
  combined_labels = np.concatenate((y_desired_class, y_other_class))

  train_data, test_data, train_labels, test_labels = train_test_split(combined_data, combined_labels, test_size=0.2)

  class_model = create_model()

  class_model.fit(
  train_data,
  train_labels,
  validation_data=(test_data, test_labels),
  epochs=10,
  batch_size=200)

  class_model.save("/content/drive/MyDrive/2 day split /ISO-DIGIT/"+str(datasetmap[normal_class])+".h5")

  print("model",normal_class,"trained",normal_class)

  # Clear the model and free up the memory
  K.clear_session()
  del class_model